![](../../images/rivacon_frontmark_combined_header.png)

# Day Counter, Roll Conventions and Schedules

In [1]:
import os, sys, holidays
from datetime import date, timedelta, datetime

# Get the current working directory
current_dir = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_dir, "..", ".."))

# Add the grandparent directory to sys.path
if grandparent_dir not in sys.path:
    sys.path.insert(0, grandparent_dir)

print("Grandparent directory containing Rivapy module added to sys.path:", grandparent_dir)
from rivapy.tools.enums import RollConvention
from rivapy.tools.datetools import Period, Schedule

Grandparent directory containing Rivapy module added to sys.path: C:\Users\Anwender\Desktop\RiVaPy


C:\Users\Anwender\.conda\envs\python_3.11\Lib\site-packages\pyvacon\__init__.py:12: UserWarning: Imported pyvacon is just beta version.
  _warnings.warn('Imported pyvacon is just beta version.')
C:\Users\Anwender\Desktop\RiVaPy\rivapy\__init__.py:9: UserWarning: Imported pyvacon is just beta version.
  warnings.warn('Imported pyvacon is just beta version.')


## Schedules
For different financial instruments such as Bonds or Swaps a vector for certain dates is needed. These vectors are usually called schedules. They may describe the dates where certain payments or other events (such as fixings) occur. Such schedules are normally based on some construction logic such as "the first Monday of each month in year 2020". The generation of these schedules is therefore often not handmade but made by algorithms, which we call schedule generators. To create such a schedule with a generator, we have three main ingredients:
- Periods describing the frequency of the dates such as monthly, yearly, quarterly etc.
- A holiday calendar since most often schedules contain only business days
- A roll convention which defines what happens if the algorithm ends at a holiday
which we discuss in the following.

### Periods
A period is described by the number of years/months/days and is the basis in the schedule generation.

In [2]:
period_1yr = Period(1,0,0) # create a period of 1 year (first argument of this method describes years, second month and last days)
period_3m = Period(0,3,0)
period_30days = Period(0,0,30)

### Holiday calendar
We use the Python library [holidays](https://holidays.readthedocs.io/en/latest/) as a Holiday calendar for determining business days.
See example using the holiday calendar of the Euopean Central Bank below.

In [3]:
ecb_holidays = holidays.ECB(2024)       # Optionally a year or a list of years can be provided. 
                                        # By default the years are automatically extended when needed, see below.

for d,n in ecb_holidays.items():
    print(d,n)
    
print(ecb_holidays.is_working_day("2023-1-1"))  # This can be used to check for working day (holidays and weekends considered) 
                                                # and adds 2023 to ecb_holidays

for d,n in ecb_holidays.items():
    print(d,n)

2024-01-01 New Year's Day
2024-03-29 Good Friday
2024-04-01 Easter Monday
2024-05-01 1 May (Labour Day)
2024-12-25 Christmas Day
2024-12-26 26 December
False
2024-01-01 New Year's Day
2024-03-29 Good Friday
2024-04-01 Easter Monday
2024-05-01 1 May (Labour Day)
2024-12-25 Christmas Day
2024-12-26 26 December
2023-01-01 New Year's Day
2023-04-07 Good Friday
2023-04-10 Easter Monday
2023-05-01 1 May (Labour Day)
2023-12-25 Christmas Day
2023-12-26 26 December


### Rolling Conventions

Rolling conventions, also known as business day adjustment rules, determine how dates are adjusted if they fall on non-business days (e.g., weekends or holidays). Common rolling conventions include:

1. **Following**  
   If the date falls on a non-business day, it is rolled forward to the next business day.

2. **Modified Following**  
   Similar to Following, but if rolling forward moves the date to the next month, it is rolled backward to the preceding business day.

3. **Preceding**  
   If the date falls on a non-business day, it is rolled backward to the previous business day.

4. **Modified Preceding**  
   Similar to Preceding, but adjustments are made to ensure the date stays in the same month.

5. **Unadjusted**  
   The date is not adjusted, even if it falls on a non-business day.

These conventions are essential in determining the exact timing of payments or other financial events.


In [4]:
# Example of selecting a Roll Convention in rivapy
RollConvention.FOLLOWING
RollConvention.MODIFIED_FOLLOWING
RollConvention.PRECEDING
RollConvention.MODIFIED_PRECEDING
RollConvention.UNADJUSTED

<RollConvention.UNADJUSTED: 'Unadjusted'>

### Schedule generation
To create a schedule we first have to create a specification containing the information described above (roll conventions, holidays, periods). We may then call the generate method to create a list of dates defining the schedule.

**Additional Options:**

- **`backwards`** *(bool, optional)*:  
  Defines the direction for rolling out the schedule.  
  - **True**: The schedule will be rolled out *backwards* (from the end day to the start day).
  - **False**: The schedule will be rolled out *forwards* (from the start day to the end day). 
  - Defaults to **True**.

- **`stub`** *(bool, optional)*:  
  Defines the behavior for the first/last period when it is shorter than the others.  
  - **True**: The first/last period is accepted even if it is shorter.  
  - **False**: The remaining days of the shorter period are added to the neighboring period.  
  - Defaults to **True**.

In [5]:
#no business day adjustments
unadjusted_schedule = Schedule._roll_out(from_=date(2024,1,1), to_=date(2024,12,31), term=period_3m, backwards=False,
                  allow_stub=False)
# Show undajusted schedule
print("Unadjusted:")
for s in unadjusted_schedule: 
    print(s.date())
print()

# adjust for business days (holidays and weekends)
schedule_2024_3m_period = Schedule(start_day = date(2024,1,1),
                                    end_day = date(2024,12,31),
                                    time_period = period_3m,
                                    backwards = False,
                                    stub = False,
                                    business_day_convention = RollConvention.MODIFIED_FOLLOWING,
                                    calendar = holidays.ECB())
adjusted_schedule = schedule_2024_3m_period.generate_dates(ends_only=False)
print("Adjusted for Working Day:")
for s in adjusted_schedule: 
    print(s.date())


Unadjusted:
2024-01-01
2024-04-01
2024-07-01
2024-12-31

Adjusted for Working Day:
2024-01-02
2024-04-02
2024-07-01
2024-12-31


### Day counter
Date conventions, or day count conventions, define how the time between two dates is measured for financial calculations. They determine how interest accrues between dates. A good overview on day count conventions as well as other market conventions can be found [here](https://www.isda.org/a/smMDE/Blackline-2000-v-2006-ISDA-Definitions.pdf). Here are some common conventions implemented in rivapy:

#### 30/360 conventions
All conventions in this class assume that each month has 30 days and that the year has 360 days. 
The general formula is therefore given by 

$$ \frac{360 (Y_{2,adj} - Y_{1,adj}) + 30(M_{2,adj}-M_{1,adj}) + D_{2,adj}-D_{1,adj}}{ 360}$$ 

where $Y_{i,adj}$ denotes the adjusted year of date $i$, $M_{i,adj}$ the month and $D_{i,adj}$ the day. 
The methods differ on how they adjust the given dates as described below.
##### 30U/360
Adjustment rules are
- $D_1=31$ then set $D_1=30$
- $D_2=31$ and $D_1=30$ or $D_1=31$ then set $D_2=30$

This convention is also called 30/360 US, 30U/360 or 360/360.


##### 30E/360
Adjustment rules are
- $D_1=31$ then set $D_1=30$
- $D_2=31$ then set $D_2=30$
which is also called Eurobond basis.

#### ACT/365 Fixed
Here, the definition is 
$$ \frac{d_2-d_1}{365}$$
where $d_2-d_1$ is the number of dates between the two dates.

#### ACT/ACT
The year fraction is computed as  $$ \frac{Days\; in\; non leap\; year}{365} + \frac{Days\; in\; leap\; year}{366}$$

In [6]:
from rivapy.tools.datetools import DayCounter as dc

# this is how you can get the year fraction based on a date count convention in rivapy:
# Options:
#   - "Act365Fixed"
#   - "ActAct"
#   - "Act360"
#   - "30U360"
#   - "30E360"
#   - "30360ISDA"
d1 = date(2024,1,1)
d2 = date(2024,7,1)

dc("ActAct").yf(d1, d2)

#printing all implemented types in rivapy

def show_day_counter(d1,d2,dc_names):
    if isinstance(dc_names, str):
        dc_names = [dc_names]
    print("d1"+":", d1)
    print("d2"+":", d2)
    for dc_name in dc_names:
        print(dc_name + ":",dc(dc_name).yf(d1,d2))
        
show_day_counter(date(2024,1,1),date(2024,7,1),["30360ISDA","30E360", "30U360","ActAct", "Act360","Act365Fixed"])


#30,42/365
#act/act ISDA
#act/act ICMA 

d1: 2024-01-01
d2: 2024-07-01
30360ISDA: 0.5
30E360: 0.5
30U360: 0.5
ActAct: 0.4972677595628415
Act360: 0.5055555555555555
Act365Fixed: 0.4986301369863014


#### ACT/ACT ICMA Day Count Convention

The **ACT/ACT ICMA** convention is commonly used in bond markets to calculate the day count fraction, particularly for determining accrued interest and bond yields. It provides a standardized way to calculate the fraction of a year represented by a given period, based on actual calendar days and the bond's coupon schedule.

$$
\text{Day Count Fraction} = \frac{d_2-d_1}{\text{Days in Coupon Period} \times \text{Number of Coupon Periods in a Year}}
$$

Where:
- **$d_2-d_1$**: is the number of dates between the two dates.
- **Days in Coupon Period**: The length of the current coupon period in days.
- **Number of Coupon Periods in a Year**: Based on the bond's coupon payment frequency (e.g., 1 for annual, 2 for semi-annual, 4 for quarterly).

**Example**:
Semi-annual bond with last coupon on
1st May, next coupon on 1 November (number of days: 184). On 31st May, the year franction is calculated as:
$$\frac{30}{184 \times 2} = \frac{30}{368}$$

In [9]:
# ACT/ACT ICMA is used for bonds and requires additional arguments for coupon period and frequency 
dc.yf_ActActICMA(date(2024,5,1), date(2024,5,31), coupon_period_in_days=184, coupon_frequency=2)

0.08152173913043478